In [1]:
import os
import pickle
import pandas as pd
from collections import defaultdict

In [77]:
#symptomatic, location

In [2]:
questions=["age","tipo","tecnica","family","history","symtomatic",
           "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
          "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]
questions=["tipo","tecnica","family",
           "prosthesis","birads","density_mammo","calcifications_benign","ganglio_mamo","density_echo","lymph_benign","lymph_suspicious","simple_cyst","ductal_ectasia","nodules_echo_num","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_known","nodules_echo_stable"]


In [3]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
def calculate_f1(pred,truth,point=False,average='macro'):
    pred_clean={}
    truth_clean={}
    for key,real in truth.items():
        if key in pred:
            
            predicted=pred[key] 
            if point==True:
                predicted=predicted.split(".")[0]+"."
            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
                
            pred_clean[key]=predicted.lower()
            truth_clean[key]=str(real).lower()
    # Get ground truth and predicted labels
    y_true = list(truth_clean.values())
    y_pred = list(pred_clean.values())
    
    # Define the valid labels based on the truth only
    valid_labels = sorted(set(y_true))
    
    # Replace out-of-scope predictions with something invalid
    # e.g., keep them but let them be counted as wrong
    y_pred_cleaned = [
        p if p in valid_labels else 'INVALID' for p in y_pred
    ]
    
    # Now align with true values
    y_true_final = []
    y_pred_final = []
    
    for yt, yp in zip(y_true, y_pred_cleaned):
        y_true_final.append(yt)
        y_pred_final.append(yp)
    
    # Compute F1 over valid labels only
    f1 = f1_score(
        y_true_final,
        y_pred_final,
        labels=valid_labels,   # Only ground-truth classes
        average='macro'        # or whatever average you want
    )
    report = classification_report(
        y_true,
        y_pred_cleaned,
        labels=valid_labels,     # Only ground truth classes
        zero_division=0,         # Avoid warnings for undefined precision/recall
        digits=4                 # Optional: better precision
    )
    print(report)
    return f1


In [4]:
def calculate_acc(pred,truth,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]    
            if point==True:

                predicted=predicted.split(".")[0]+"."

            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
            if predicted.lower()==str(real).lower():
                acc+=1
            else:
                print(key,real,predicted)
        else:
            print(key,real)
        # else:
        #     print(key)
    return acc/total

In [5]:
from sklearn.metrics import classification_report, accuracy_score
accuracies=defaultdict(list)
f1s=defaultdict(list)
models=["biogpt","clinicalt5","clinicalt5_tokenized", "biogpt_no_info","biogpt_answer"]
for model in models:
    print(model)
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for tipo in questions:
        
        print(tipo)
        with open(f"test_results/truth_dic/{tipo}.pkl", "rb") as file:
                truth=pickle.load(file)
        with open(f"test_results/results_dic_{tipo}/{model}.pkl", "rb") as file:
                output=pickle.load(file)
        if tipo=="family":
            print(len(output))
        if tipo=="birads":
            truth["746-766-264-20240605-182548_birads"]="BI-RADS 2."
        if model=="biogpt" and tipo=="nodules_echo_echogenicity":
            
            output["893-892-465-20230313-152447_nodules_echo_echogenicity_1"]="hypoechoic."
        if tipo!="nodules_echo_size":
            acc=calculate_acc(output,truth,point=True)
            
            f1=calculate_f1(output,truth,point=True)
            if tipo=="density_echo":
                print(truth,output)
                print(f1)
                print(len(truth),len(output))
        else:
            acc=calculate_acc(output,truth,point="mm")
            f1=calculate_f1(output,truth,point="mm")
        accuracies[tipo].append(acc)
        f1s[tipo].append(f1)



biogpt
tipo
746-766-264-20240304-171357_tipo normal control or revision report. biopsy report.
                                    precision    recall  f1-score   support

                    biopsy report.     0.8333    1.0000    0.9091         5
  nodal staging ultrasound report.     1.0000    1.0000    1.0000         1
normal control or revision report.     1.0000    0.9844    0.9921        64

                          accuracy                         0.9857        70
                         macro avg     0.9444    0.9948    0.9671        70
                      weighted avg     0.9881    0.9857    0.9863        70

tecnica
                             precision    recall  f1-score   support

mammography and ultrasound.     1.0000    1.0000    1.0000        53
     only ultrasound study.     1.0000    1.0000    1.0000        17

                   accuracy                         1.0000        70
                  macro avg     1.0000    1.0000    1.0000        70
               

In [6]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo_num,nodules_echo_shape,nodules_echo_margin,nodules_echo_echogenicity,nodules_echo_known,nodules_echo_stable,average
model,,,,,,,,,,,,,,,,,,,,
biogpt,0.985714,1.000000,0.968750,1.00000,0.984375,1.00000,0.984375,0.984375,0.984375,0.9375,0.984375,1.0000,0.984375,0.937500,0.88,0.96,0.96,0.80,0.923077,0.960989
clinicalt5,0.957143,0.985714,0.906250,0.96875,0.421875,0.93750,0.734375,0.953125,0.890625,0.8750,0.984375,0.8750,0.953125,0.640625,0.88,0.84,0.88,0.48,0.769231,0.838564
clinicalt5_tokenized,0.942857,0.971429,0.921875,1.00000,0.921875,0.93750,0.953125,0.984375,0.906250,0.8750,0.984375,0.9375,0.984375,0.875000,0.88,0.80,0.96,0.76,0.769231,0.913935
biogpt_no_info,0.985714,0.971429,0.953125,1.00000,0.984375,0.96875,0.984375,0.984375,0.979592,0.9375,1.000000,1.0000,0.984375,0.906250,0.84,0.92,0.96,0.84,0.923077,0.953839
biogpt_answer,0.985714,1.000000,0.968750,1.00000,0.984375,1.00000,0.984375,0.984375,0.984375,0.9375,0.984375,1.0000,0.984375,0.937500,0.88,0.96,0.92,0.80,0.923077,0.958884


In [7]:
biogpt_acc = list(data.loc["biogpt",:])[:-1]
biogpt_acc

[0.9857142857142858,
 1.0,
 0.96875,
 1.0,
 0.984375,
 1.0,
 0.984375,
 0.984375,
 0.984375,
 0.9375,
 0.984375,
 1.0,
 0.984375,
 0.9375,
 0.88,
 0.96,
 0.96,
 0.8,
 0.9230769230769231]

In [23]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo_num,nodules_echo_shape,nodules_echo_margin,nodules_echo_echogenicity,nodules_echo_known,nodules_echo_stable,average
model,,,,,,,,,,,,,,,,,,,,
biogpt,0.967072,1.000000,0.699519,1.000000,0.950249,1.000000,0.984279,0.895935,0.987698,0.839599,0.829333,1.000000,0.924439,0.934359,0.899335,0.905797,0.738095,0.800000,0.811594,0.903542
clinicalt5,0.575700,0.980176,0.611742,0.492063,0.435584,0.948434,0.702326,0.488000,0.897629,0.632184,0.829333,0.871872,0.488000,0.460213,0.852808,0.458095,0.620952,0.324324,0.783333,0.655409
clinicalt5_tokenized,0.516176,0.975415,0.876716,1.000000,0.940380,0.941482,0.953022,0.895935,0.904015,0.741414,0.829333,0.937439,0.924439,0.868718,0.920000,0.428704,0.738095,0.759615,0.434783,0.820299
biogpt_no_info,0.967072,0.959491,0.678571,1.000000,0.950249,0.971487,0.984279,0.895935,0.993243,0.839599,1.000000,1.000000,0.924439,0.902834,0.804226,0.644928,0.738095,0.839744,0.876190,0.893178
biogpt_answer,0.967072,1.000000,0.699519,1.000000,0.950249,1.000000,0.984279,0.895935,0.987698,0.839599,0.829333,1.000000,0.924439,0.934359,0.899335,0.905797,0.727226,0.800000,0.811594,0.902970


In [10]:
biogpt_f1 = list(data.loc["biogpt",:])[:-1]

In [11]:
TIPO=["biopsy report", "nodal staging ultrasound report", "normal control or revision report"]
TECNICA=["only ultrasound study", "only mammography study", "mammography and ultrasound"]
FAMILY=["no family history", "first degree", "second degree"]
PROSTHESIS=["no prosthesis","yes prosthesis"]
BIRADS=["BI-RADS 0","BI-RADS 1","BI-RADS 2","BI-RADS 3","BI-RADS 4A","BI-RADS 4B","BI-RADS 4C","BI-RADS 5","BI-RADS 6"]
DENSITY_MAMMO=["ACR A","ACR B","ACR C","ACR D","unknown density mammo"]
CALCIFICATIONS_BENIGN=["no calcifications","yes calcifications"]
GANGLIO_MAMO=["no ganglio","yes ganglio"]
DENSITY_ECHO=["homogeneous fibroglandular","heterogeneous fibroglandular","fibroglandular and fat","homogeneous fatty","unknown density echo"]
LYMPH_BENIGN=["no lymph benign","yes lymph benign"]
LYMPH_SUSPICIOUS=["no lymph suspicious","yes lymph suspicious"]
SIMPLE_CYST=["no cyst","yes cyst"]
DUCTAL_ECTASIA=["no ectasia","yes ectasia"]
NODULES_ECHO=["no nodules", "yes nodules"]
NODULES_SHAPE=["oval","round","lobulated","irregular","unknown shape"]
NODULES_MARGIN=["circumscribed","spiculated","indistinct","not circumscribed","unknown margin"]
NODULES_ECHOGENICITY=["hypoechoic", "isoechoic", "heterogeneous","complex cystic and solid","unknown echogenicity"]
NODULES_KNOWN=["no known", "yes known"]
NODULES_STABLE=["grown stable","shrunk stable", "yes stable"]

DICTIONARY={"tipo":TIPO,"tecnica":TECNICA,"family":FAMILY,"prosthesis":PROSTHESIS,"birads":BIRADS,"density_mammo":DENSITY_MAMMO,"calcifications_benign":CALCIFICATIONS_BENIGN,
            "ganglio_mamo":GANGLIO_MAMO,"density_echo":DENSITY_ECHO,"lymph_benign":LYMPH_BENIGN,"lymph_suspicious":LYMPH_SUSPICIOUS,"simple_cyst":SIMPLE_CYST,"ductal_ectasia":DUCTAL_ECTASIA,
           "nodules_echo": NODULES_ECHO,"nodules_shape":NODULES_SHAPE,"nodules_margin":NODULES_MARGIN, "nodules_echogenicity":NODULES_ECHOGENICITY, "nodules_known":NODULES_KNOWN, "nodules_stable":NODULES_STABLE}

def evaluate_per_question(predicted, tested, DICTIONARY, average="macro"):
    """
    Evaluate model predictions per question type.
    
    Parameters:
    - predicted: array of predicted label indices (flattened from all folds)
    - tested: array of true label indices (same shape as predicted)
    - DICTIONARY: dict mapping each question to its list of class names
    """
    
    # Step 1: Build global index → (question, class_name) mapping
    idx_to_question_value = {}
    offset = 0
    question_offsets = {}
    for question, class_list in DICTIONARY.items():
        question_offsets[question] = offset
        for i, label in enumerate(class_list):
            idx_to_question_value[offset + i] = (question, label)
        offset += len(class_list)

    # Step 2: Group predictions by question
    per_question_true = defaultdict(list)
    per_question_pred = defaultdict(list)

    for true_idx, pred_idx in zip(tested, predicted):
        q_true, _ = idx_to_question_value[true_idx]
        # You can check if q_true == q_pred here for safety if needed
        per_question_true[q_true].append(int(true_idx))
        per_question_pred[q_true].append(int(pred_idx))

    # Step 3: Classification reports

    f1s={}
    accuracies={}
    for question, true_labels in per_question_true.items():
        pred_labels = per_question_pred[question]
        label_names = DICTIONARY[question]
        start = question_offsets[question]
        end = start + len(label_names)
        question_label_ids = list(range(start, end))
        # print(question)
        #Añadimos el family history third que los BERT modelos no pueden capturar
        print(len(true_labels))
        if question=="family":
            print(len(true_labels))
            true_labels=[200]+true_labels
            pred_labels=[7]+pred_labels
            print(len(true_labels))
        
        f1=f1_score(true_labels,pred_labels,labels=sorted(set(true_labels)),average=average)
        f1s[question]=f1
        accuracy=accuracy_score(true_labels,pred_labels)
        accuracies[question]=accuracy
    return f1s,accuracies        


In [12]:
import re
import numpy as np
models=["predicted_biobert","predicted_biomed","predicted_bluebert"]
accuracies=defaultdict(list)
f1s=defaultdict(list)
for model in models:
    model_truth=re.sub("predicted","tested",model)
    truth=np.load(f"test_results/{model_truth}_classification.npy")
    outputs=np.load(f"test_results/{model}_classification.npy")
    print(truth)
    
    f1s_model,accuracies_model=evaluate_per_question(outputs,truth,DICTIONARY)
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for question in DICTIONARY.keys():
        accuracies[question].append(accuracies_model[question])
        f1s[question].append(f1s_model[question])
        

[ 2  3  7 ... 34 40 42]
70
70
63
63
64
64
64
64
64
64
64
64
64
64
64
64
25
25
25
25
13
[ 2  3  7 ... 34 40 42]
70
70
63
63
64
64
64
64
64
64
64
64
64
64
64
64
25
25
25
25
13
[ 2  3  7 ... 34 40 42]
70
70
63
63
64
64
64
64
64
64
64
64
64
64
64
64
25
25
25
25
13


In [13]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo,nodules_shape,nodules_margin,nodules_echogenicity,nodules_known,nodules_stable,average
model,,,,,,,,,,,,,,,,,,,,
predicted_biobert,0.939153,0.980952,0.747475,1.0,0.837154,0.963429,0.952837,0.488000,0.894757,0.785953,0.829333,0.984341,0.924439,0.951698,0.640168,0.623106,0.693452,0.800000,0.458333,0.815504
predicted_biomed,0.967072,1.000000,0.725769,1.0,0.837154,1.000000,0.968627,0.741935,0.987698,0.679198,0.829333,0.984341,0.924439,0.935223,0.568516,0.633070,0.715000,0.839744,0.458333,0.831340
predicted_bluebert,0.928125,0.942857,0.711577,1.0,0.425770,0.906154,0.904762,0.488000,0.860079,0.735693,0.829333,0.952837,0.924439,0.856538,0.533333,0.403704,0.445055,0.839744,0.409091,0.741952


In [14]:
biomed_f1 = list(data.loc["predicted_biomed",:])[:-1]

In [15]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo,nodules_shape,nodules_margin,nodules_echogenicity,nodules_known,nodules_stable,average
model,,,,,,,,,,,,,,,,,,,,
predicted_biobert,0.971429,0.985714,0.984375,1.0,0.96875,0.96875,0.953125,0.953125,0.937500,0.921875,0.984375,0.984375,0.984375,0.953125,0.84,0.88,0.88,0.80,0.846154,0.936687
predicted_biomed,0.985714,1.000000,0.968750,1.0,0.96875,1.00000,0.968750,0.968750,0.984375,0.875000,0.984375,0.984375,0.984375,0.937500,0.72,0.88,0.92,0.84,0.846154,0.937730
predicted_bluebert,0.971429,0.957143,0.968750,1.0,0.71875,0.90625,0.906250,0.953125,0.890625,0.890625,0.984375,0.953125,0.984375,0.859375,0.72,0.76,0.84,0.84,0.692308,0.884027


In [16]:
biomed_acc = list(data.loc["predicted_biomed",:])[:-1]

In [20]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_acc, biomed_acc, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_f1, biomed_f1, alternative='greater')

print(f"Wilcoxon statistic f1: {statistic}")
print(f"p-value: {p_value}")

NameError: name 'biomed_acc' is not defined

In [21]:
biogpt=[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1677,
 1.0,
 1.0,
 0.0019,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2638,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8763,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3209,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4092,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8641,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5016,
 0.1649,
 1.0,
 1.0,
 1.0,
 0.252,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0623,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9951,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8368,
 1.0,
 0.3967,
 1.0,
 1.0,
 0.9867,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.489]

In [26]:
biogpt_no_age=[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1677,
 1.0,
 1.0,
 0.0019,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2638,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8763,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3209,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4092,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8641,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5016,
 0.1649,
 1.0,
 1.0,
 1.0,
 0.252,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0623,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9951,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8368,
 1.0,
 0.3967,
 1.0,
 1.0,
 0.9867,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.489]

In [23]:
BioMedBERT=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7013, 1.0, 1.0, 1.0, 0.1352, 0.1352, 1.0, -0.0418, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0964, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7602, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4887, 1.0, 0.1503, 1.0, 1.0, 1.0, 1.0, 0.7231, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8368, 1.0, 0.3967, 1.0, 1.0, 0.991, 1.0, 1.0, 0.5678, 1.0, 1.0, 1.0, 1.0]

In [24]:
BioMedBERT_no_age=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7013, 1.0, 1.0, 1.0, 0.1352, 0.1352, 1.0, -0.0418, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0964, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7602, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4887, 1.0, 0.1503, 1.0, 1.0, 1.0, 1.0, 0.7231, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8368, 1.0, 0.3967, 1.0, 1.0, 0.991, 1.0, 1.0, 0.5678, 1.0, 1.0, 1.0, 1.0]

In [27]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(BioMedBERT,biogpt, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(BioMedBERT_no_age,biogpt_no_age, alternative='greater')

print(f"Wilcoxon statistic f1: {statistic}")
print(f"p-value: {p_value}")

Wilcoxon statistic: 105.0
p-value: 0.3436718201761428
Wilcoxon statistic f1: 105.0
p-value: 0.3436718201761428
